In [3]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import multiprocessing as mp
from joblib import Parallel, delayed
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from sqlalchemy import create_engine

In [4]:
opciones=Options()
opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.add_argument('--incognito')
opciones.add_argument('--start-maximized')

In [5]:
url = 'https://www.rfevb.com/calendario-superliga-masculina-2023-2024'
driver = webdriver.Chrome(options = opciones)
driver.get(url)

In [6]:
# Click cookies
driver.find_element(By.XPATH , '//*[@id="lnkCookie"]').click()

In [7]:
# Seleccionamos todas las jornadas de la liga.
driver.find_element(By.XPATH,'//*[@id="jornadas"]/option[1]').click()

In [8]:
# Extrameos datos de la caja principal de jornadas.

data = driver.find_element(By.XPATH,'//*[@id="CPContenido_rptPlugins_PanPlugin_0"]/div').text.split('\n')[52:]

In [9]:
len(data)

550

In [10]:
data

['JORNADA 1',
 '1',
 '07/10/23',
 '18:30',
 'Unicaja Costa de Almería 3 - 1 UPV Leleman Conqueridor',
 '2',
 '07/10/23',
 '19:00',
 'Club Vóley Palma 2 - 3 Pamesa Teruel Voleibol',
 '3',
 '07/10/23',
 '19:00',
 'Volei Villena Petrer 0 - 3 CV Melilla',
 '4',
 '08/10/23',
 '12:00',
 'CV San Roque 0 - 3 CV Guaguas',
 '5',
 '07/10/23',
 '19:00',
 'Cisneros Alter Tenerife 3 - 0 Arenal Emevé',
 '6',
 '07/10/23',
 '19:30',
 'Grupo Herce Soria 3 - 0 Conectabalear CV Manacor',
 'JORNADA 2',
 '7',
 '14/10/23',
 '18:00',
 'Conectabalear CV Manacor 0 - 3 Unicaja Costa de Almería',
 '8',
 '14/10/23',
 '17:30',
 'Arenal Emevé 1 - 3 Grupo Herce Soria',
 '9',
 '14/10/23',
 '12:00',
 'CV Guaguas 3 - 2 Cisneros Alter Tenerife',
 '10',
 '15/10/23',
 '17:00',
 'CV Melilla 3 - 0 CV San Roque',
 '11',
 '14/10/23',
 '18:30',
 'Pamesa Teruel Voleibol 3 - 0 Volei Villena Petrer',
 '12',
 '14/10/23',
 '19:00',
 'UPV Leleman Conqueridor 3 - 1 Club Vóley Palma',
 'JORNADA 3',
 '13',
 '18/10/23',
 '18:30',
 'Unica

In [11]:
jornadas = []
for i in data:
    
    if 'JORNADA'in  i:
        temp = []
        temp.append(i)
        jornadas.append(temp)
    else:
        temp.append(i)
len(jornadas)

22

# Procedemos a crear los df de las jornadas.


In [31]:
import pandas as pd

# Creo un diccionario para guardar dataframes.
dataframes_por_jornada = {}

for jornada in jornadas:
    fechas = []
    horas = []
    enfrentamientos = []
    
    for i in range(1, len(jornada), 4):
        fecha = jornada[i + 1]
        hora = jornada[i + 2]
        enfrentamiento = jornada[i + 3]

        fechas.append(fecha)
        horas.append(hora)
        enfrentamientos.append(enfrentamiento)

    # Creo DataFrame por jornada
    df = pd.DataFrame({'fecha': fechas, 'hora': horas, 'enfrentamiento': enfrentamientos})
    
    # Obtener el número de jornada
    num_jornada = int(jornada[0].split()[-1])
    
    # Almacenar el DataFrame en el diccionario
    dataframes_por_jornada[num_jornada] = df

# Combinar todos los DataFrames en uno solo
df_final = pd.concat(dataframes_por_jornada.values(), ignore_index=True)

df_final


,fecha,hora,enfrentamiento
0,07/10/23,18:30,Unicaja Costa de Almería 3 - 1 UPV Leleman Con...
1,07/10/23,19:00,Club Vóley Palma 2 - 3 Pamesa Teruel Voleibol
2,07/10/23,19:00,Volei Villena Petrer 0 - 3 CV Melilla
3,08/10/23,12:00,CV San Roque 0 - 3 CV Guaguas
4,07/10/23,19:00,Cisneros Alter Tenerife 3 - 0 Arenal Emevé
...,...,...,...
127,16/03/24,18:30,Cisneros Alter Tenerife 0 - 0 Conectabalear CV...
128,16/03/24,18:30,CV San Roque 0 - 0 Arenal Emevé
129,16/03/24,19:30,Volei Villena Petrer 0 - 0 CV Guaguas
130,16/03/24,19:30,Club Vóley Palma 0 - 0 CV Melilla
